<a href="https://colab.research.google.com/github/PUBPOL-2130/notebooks/blob/main/future/precincts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to precincts

Here, we'll pull in a precinct shapefile.  (Currently sourced to Ben Rosenblatt, [link](https://www.benjrosenblatt.com/new-york-2022-election-district-level-data).)  The point of using precincts is to be able to join election results.

In [ ]:
!curl -OL https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week3-NY-precincts.zip
!unzip week3-NY-precincts.zip

In [ ]:
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_block_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_block_with_race_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_block_with_race_populated_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_tract_gdf.zip"
!curl -OL "https://github.com/PUBPOL-2130/notebooks/raw/refs/heads/main/data/week4_36_county_tract_with_acs_gdf.zip"


!unzip week4_36_county_block_gdf.zip
!unzip week4_36_county_block_with_race_gdf.zip
!unzip week4_36_county_block_with_race_populated_gdf.zip
!unzip week4_36_county_tract_gdf.zip
!unzip week4_36_county_tract_with_acs_gdf.zip

In [ ]:
ny_precinct_gdf = gpd.read_file("week3-NY-precincts")

In [ ]:
ny_precinct_gdf

In [ ]:
sen_columns = [col for col in ny_precinct_gdf if col.startswith("USSen_")]

In [ ]:
ny_precinct_gdf["USSen_total"] = ny_precinct_gdf[sen_columns].sum(axis=1)
sen_columns.append("USSen_total")

In [ ]:
# there was an invalid geometry error in this shapefile, and this was a quick fix
county_precinct_gdf = ny_precinct_gdf[ny_precinct_gdf.CountyFP == county_fips]
county_precinct_gdf.geometry = county_precinct_gdf.geometry.buffer(0)

OK, now we'll take a look at the votes by party.  Chuck Schumer ran on both the Dem and Working Parties line; Joe Pinion challenged him as both a Republican and Constitutional Party nominee; and Diane Sare picked up a handful of votes on the LaRouche Party line.  (Look up Lyndon LaRouche, it's quite a story!)

In [ ]:
county_precinct_gdf

In [ ]:
fig, ax = plt.subplots(figsize=(40, 20))
ax.set_title(f"{county_name} (precincts)", fontsize=18)
ax.axis('off')
county_precinct_gdf.plot(ax=ax, edgecolor="0.1", linewidth=1, color="#e1f1fd")
plt.axis('off')
plt.show()

The MAUP package will let us query the blocks and figure out what precinct they are in.  That way, we can take stats on the blocks and aggregate them up to precincts -- this will let us compare race to voting.

In [ ]:
# maup.assign will take each block and figure out what precinct it is in
block_to_precinct_assignment = maup.assign(county_block_gdf, county_precinct_gdf)
block_to_precinct_assignment = block_to_precinct_assignment[~pd.isna(block_to_precinct_assignment)].astype(int)
block_to_precinct_assignment

In [ ]:
pop_weights = (
    county_block_with_race_populated_gdf["total"]
    / block_to_precinct_assignment.map(county_block_with_race_populated_gdf["total"].groupby(block_to_precinct_assignment).sum())
).fillna(0)

In [ ]:
pop_weights

In [ ]:
aggregated_votes_df = maup.prorate(block_to_precinct_assignment, county_precinct_gdf[sen_columns], weights=pop_weights)

In [ ]:
aggregated_votes_df

In [ ]:
county_block_with_election_gdf = county_block_with_race_gdf.join(aggregated_votes_df[["USSen_DEM","USSen_WOR","USSen_total"]])
county_block_with_election_gdf["sen_demwor_pct"] = (
    100 * (county_block_with_election_gdf["USSen_DEM"]+county_block_with_election_gdf["USSen_WOR"])
    / county_block_with_election_gdf["USSen_total"]
)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.axis('off')
ax.set_title(f"{county_name} 2022 Senate Dem+WorkingFam % (disaggregated to blocks)", fontsize=18)
county_block_with_election_gdf[~pd.isna(county_block_with_election_gdf["sen_demwor_pct"])].plot(
    ax=ax,
    column="sen_demwor_pct",
    vmin=0,
    vmax=100,
    **choropleth_style,
)
county_precinct_gdf.boundary.plot(
    ax=ax,
    edgecolor="0.1",
    linewidth=1.5,
)
plt.show()

In [ ]:
aggregated_votes_df["tract"] = aggregated_votes_df.index.str.slice(0, 11)

In [ ]:
aggregated_tract_votes_df = aggregated_votes_df.groupby("tract")[sen_columns].sum()
aggregated_tract_votes_df

In [ ]:
aggregated_tract_votes_df["sen_demwor_pct"] = (
    100 * aggregated_tract_votes_df[["USSen_DEM","USSen_WOR"]].sum(axis=1)
    / aggregated_tract_votes_df["USSen_total"]
)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.axis('off')
ax.set_title(f"{county_name} 2022 Senate Dem+WorkingFam % (aggregated up to tracts)", fontsize=18)
county_tract_gdf.join(aggregated_tract_votes_df).plot(
    ax=ax,
    column="sen_demwor_pct",
    vmin=0,
    vmax=100,
    **choropleth_style,
)
plt.show()

Kind of interesting that the southern half of Brooklyn doesn't seem to have supported Chuck Schumer.  Let's see if that's predicted by age or income.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 8))
axes[0].axis('off')
axes[0].set_title(f"{county_name}: median age", fontsize=10)

county_tract_with_acs_gdf.plot(
    ax=axes[0],
    column="median_age",
    vmin=0,
    **choropleth_style,
)

axes[1].axis('off')
axes[1].set_title(f"{county_name}: median income ($)", fontsize=10)
county_tract_with_acs_gdf.plot(
    ax=axes[1],
    column="median_income",
    vmin=0,
    **choropleth_style,
)

axes[2].axis('off')
axes[2].set_title(f"{county_name}: 2022 Senate Dem+WF %", fontsize=10)
county_tract_gdf.join(aggregated_tract_votes_df).plot(
    ax=axes[2],
    column="sen_demwor_pct",
    **choropleth_style,
)
plt.show()

Not really -- how about race?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 8))
axes[0].axis('off')
axes[0].set_title(f"{county_name}: WPOP/TOTPOP", fontsize=10)

county_tract_with_acs_gdf.plot(
    ax=axes[0],
    column="white_pct",
    vmin=0,
    **choropleth_style,
)



axes[1].axis('off')
axes[1].set_title(f"{county_name}: 2022 Senate Dem+WF %", fontsize=10)
county_tract_gdf.join(aggregated_tract_votes_df).plot(
    ax=axes[1],
    column="sen_demwor_pct",
    **choropleth_style,
)
plt.show()